In [2]:
import torch

from MLP_model import *
import scanpy as sc
import numpy as np
import pandas as pd
from tqdm import tqdm
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

2022-02-07 20:44:50.446364: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-02-07 20:44:50.462841: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7f924aaf3170 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-02-07 20:44:50.462852: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version


In [3]:
# select the highly expressed genes or HVGs
adata = sc.read_h5ad("/Users/kenongsu/Dropbox/Mingyao/MultiSlices/tests/HER2ST/allData_ann2.h5ad")
'''select the genes based on hvgs
sc.pp.highly_variable_genes(adata, n_top_genes=100, batch_key="pats")
hvgs = adata.var.index[adata.var["highly_variable"] == True]
geneIds = [i for i, v in enumerate(adata.var_names) if v in hvgs]
'''

# select high expression genes
testSamples = ["A1", "C1", "D1", "E1", "F1", "G1", "H1"]
allMeans = np.zeros((len(testSamples), adata.shape[1]))
for j in range(len(testSamples)):
    testSample = testSamples[j]
    sampleids = [i for i, v in enumerate(adata.obs['samples']) if v == testSample]
    oneAdata = adata[sampleids, :]
    oneMeans = oneAdata.X.mean(axis=0)
    allMeans[j] = oneMeans

allMeansMs = allMeans.mean(axis=0)
ixs = np.argsort(allMeansMs)[::-1]
geneIds = ixs[:1000]

Observation names are not unique. To make them unique, call `.obs_names_make_unique`.


In [4]:
'''main training process
Loop through each test sample
    for testSample in testSamples:
        sampleids = [i for i, v in enumerate(adata.obs['samples']) if v == testSample]
        aids = [i for i, v in enumerate(adata.obs['samples']) if v != testSample]
Here, use B1 as example
'''
testSample = "B1"
sampleids = [i for i, v in enumerate(adata.obs['samples']) if v == testSample]
aids = [i for i, v in enumerate(adata.obs['samples']) if v != testSample]
trainData = adata[aids, :]
testData = adata[sampleids, :]

In [5]:
''' preparing input for the HOPE2Net model
HOPE2Net takes 2 input: ViT features (by default) and PE (the same as the ViT features).
One can customize the image features to ResNet features.
'''
trainX1 = trainData.obsm["ViT"]
trainX2 = trainData.obsm["PE"]
trainY = trainData.X[:, geneIds]

testX1 = testData.obsm["ViT"]
testX2 = testData.obsm["PE"]
testY = testData.X[:, geneIds]

n_feature = testX1.shape[1]
n_gene = testY.shape[1]
n_sample = trainX1.shape[0]
r1s = []; r2s = []

In [6]:
'''train as a whole
To account for the gene-gene dependency, we train the model as a whole.
pytorch lightning provides EarlyStopping criterion to avoid overfitting: here we monitor the validation loss.
The best strategy is to use a small learning rate and keep track with validation loss (with patience). Also, we can decay the learning rate.
For validation dataset, we split the training samples into 90% training data and 10% validation data (randomly).
'''
pl.seed_everything(123)
n_epoch = 1000
batch_size = 16
Trdataset = TrDataset(X1=torch.Tensor(trainX1), X2=torch.Tensor(trainX2), Y=torch.Tensor(trainY).view(n_sample, n_gene))
nval = int(len(trainY) * 0.2)
ntrain = len(trainY) - nval
train_set, val_set = random_split(Trdataset, [ntrain, nval])
Train_dataloader = DataLoader(train_set, batch_size=batch_size, shuffle=True)
Val_dataloader = DataLoader(val_set, batch_size=batch_size, shuffle=False)
early_stop_callback = EarlyStopping(monitor="val_loss", min_delta=1e-5, patience=6, verbose=True, mode="min")
trainer = pl.Trainer(max_epochs=1000, callbacks=[early_stop_callback])
plmodel = plNet(n_feature=768, n_hidden1=64, n_hidden2=32, n_output=n_gene, learning_rate=1e-5)
trainer.fit(plmodel, train_dataloaders=Train_dataloader, val_dataloaders=Val_dataloader)

Global seed set to 123
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs

  | Name    | Type   | Params
-----------------------------------
0 | hidden1 | Linear | 49.2 K
1 | hidden2 | Linear | 2.1 K 
2 | predict | Linear | 33.0 K
-----------------------------------
85.8 K    Trainable params
0         Non-trainable params
85.8 K    Total params
0.343     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

/Users/kenongsu/opt/anaconda3/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:105: UserWarning: The dataloader, val dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
Global seed set to 123
/Users/kenongsu/opt/anaconda3/lib/python3.8/site-packages/pytorch_lightning/trainer/data_loading.py:105: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: -1it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Monitored metric val_loss did not improve in the last 6 records. Best score: 7.318. Signaling Trainer to stop.


In [7]:
''' Check the model accuracy'''
trainYpreds = plmodel(torch.Tensor(trainX1), torch.Tensor(trainX2))
testYpreds = plmodel(torch.Tensor(testX1), torch.Tensor(testX2))
from scipy.stats import pearsonr
for i in tqdm(range(n_gene)):
    r1, _ = pearsonr(trainYpreds.detach().numpy()[:,i], trainY[:, i])
    r2, _ = pearsonr(testYpreds.detach().numpy()[:,i], testY[:, i])
    r1s.append(r1)
    r2s.append(r2)

print(testSample, np.mean(r2s), np.max(r2s))

100%|██████████| 1000/1000 [00:00<00:00, 1657.55it/s]

B1 0.35316666580871103 0.6579763284579242


In [12]:
plmodel.on_save_checkpoint("plmodel_B1.pt")

In [13]:
torch.save(plmodel, "plmodel_B1.pt")